In [2]:
!pip install colpali_engine==0.1.1
!pip install mteb
!pip install PyMuPDF

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 14.4 MB/s eta 0:00:00
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=dcc3dcfb3860ecd6d53dc764ce3131516dc39516b1d15092189b8ffbf5e110bf
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 20.5 MB/s eta 0:00:00


In [7]:
import os
import sys
import torch
import typer
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoProcessor
from colpali_engine.models.paligemma_colbert_architecture import ColPali
from colpali_engine.trainer.retrieval_evaluator import CustomEvaluator
from colpali_engine.utils.colpali_processing_utils import process_queries
from PIL import Image
import fitz

In [2]:
# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    """Extracts text from a single PDF."""
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text("text")  # Get raw text
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return None

In [3]:
# Step 2: Process all PDFs in the specified folder
def process_pdf_folder(folder_path):
    """Process all PDF files in a folder and return their extracted texts."""
    pdf_texts = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, file_name)
            print(f"Extracting text from {file_name}...")
            text = extract_text_from_pdf(pdf_path)
            if text:
                pdf_texts[file_name] = text
    return pdf_texts

In [4]:
# Step 3: Create document embeddings
def create_document_embeddings(texts, processor, model):
    """Create embeddings for all extracted texts from PDFs."""
    dataloader = DataLoader(
        texts,
        batch_size=4,
        shuffle=False,
        collate_fn=lambda x: process_queries(processor, x, Image.new("RGB", (448, 448), (255, 255, 255))),
    )

    embeddings = []
    # passing through tqdm to track progress
    for batch_text in tqdm(dataloader):
        with torch.no_grad():
            batch_text = {k: v.to(model.device) for k, v in batch_text.items()}
            embeddings_text = model(**batch_text)
        embeddings.extend(list(torch.unbind(embeddings_text.to("cpu"))))
    return embeddings


In [5]:
# Step 4: Keep conversation history
class ConversationManager:
    def __init__(self):
        self.history = []

    def add_to_history(self, question, answer):
        """Add a new question-answer pair to the conversation history."""
        self.history.append((question, answer))

    def get_contextual_query(self, new_question):
        """Generate a new query by appending the conversation history."""
        if self.history:
            # Build the new query by appending previous questions and answers to the current question
            context = "\n".join([f"Q: {q}\nA: {a}" for q, a in self.history])
            return f"{context}\nFollow-up: {new_question}"
        else:
            return new_question


In [8]:
import os
import torch
import typer
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoProcessor
from colpali_engine.models.paligemma_colbert_architecture import ColPali
from colpali_engine.trainer.retrieval_evaluator import CustomEvaluator
from colpali_engine.utils.colpali_processing_utils import process_queries
from PIL import Image
import fitz


# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    """Extracts text from a single PDF."""
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text("text")  # Get raw text
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return None


# Step 2: Process all PDFs in the specified folder
def process_pdf_folder(folder_path):
    """Process all PDF files in a folder and return their extracted texts."""
    pdf_texts = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, file_name)
            print(f"Extracting text from {file_name}...")
            text = extract_text_from_pdf(pdf_path)
            if text:
                pdf_texts[file_name] = text
    return pdf_texts


# Step 3: Create document embeddings
def create_document_embeddings(texts, processor, model):
    """Create embeddings for all extracted texts from PDFs."""
    dataloader = DataLoader(
        texts,
        batch_size=4,
        shuffle=False,
        collate_fn=lambda x: process_queries(processor, x, Image.new("RGB", (448, 448), (255, 255, 255))),
    )

    embeddings = []
    for batch_text in tqdm(dataloader):
        with torch.no_grad():
            batch_text = {k: v.to(model.device) for k, v in batch_text.items()}
            embeddings_text = model(**batch_text)
        embeddings.extend(list(torch.unbind(embeddings_text.to("cpu"))))
    return embeddings


# Step 4: Keep conversation history
class ConversationManager:
    def __init__(self):
        self.history = []

    def add_to_history(self, question, answer):
        """Add a new question-answer pair to the conversation history."""
        self.history.append((question, answer))

    def get_contextual_query(self, new_question):
        """Generate a new query by appending the conversation history."""
        if self.history:
            # Build the new query by appending previous questions and answers to the current question
            context = "\n".join([f"Q: {q}\nA: {a}" for q, a in self.history])
            return f"{context}\nFollow-up: {new_question}"
        else:
            return new_question


# Step 5: Main function to run the pipeline with memory
def main(folder_path: str):
    # Load the ColPali model
    model_name = "vidore/colpali"
    model = ColPali.from_pretrained("vidore/colpaligemma-3b-mix-448-base", torch_dtype=torch.bfloat16, device_map="cuda").eval()
    model.load_adapter(model_name)
    processor = AutoProcessor.from_pretrained(model_name)

    # Process PDF files and extract text
    print("Processing PDFs...")
    pdf_texts = process_pdf_folder(folder_path)

    if not pdf_texts:
        print("No PDFs processed or empty PDF folder.")
        return

    # Create embeddings for the documents
    doc_texts = list(pdf_texts.values())  # Get all the extracted texts
    print("Creating document embeddings...")
    doc_embeddings = create_document_embeddings(doc_texts, processor, model)

    # Prepare the evaluation engine and conversation manager
    retriever_evaluator = CustomEvaluator(is_multi_vector=True)
    conversation_manager = ConversationManager()

    # Interactive question-answer loop
    print("Ready to answer questions! Type 'exit' to stop.")
    while True:
        query = input("Enter your question: ")
        if query.lower() == "exit":
            break

        # Use conversation history to generate contextual query
        contextual_query = conversation_manager.get_contextual_query(query)

        # Create query embedding
        query_embedding = create_document_embeddings([contextual_query], processor, model)

        # Evaluate and find the best matching document
        scores = retriever_evaluator.evaluate(query_embedding, doc_embeddings)
        best_doc_idx = scores.argmax(axis=1)[0]

        # Show the best match from the PDF texts
        best_doc_name = list(pdf_texts.keys())[best_doc_idx]
        best_doc_text = doc_texts[best_doc_idx][:500]  # Show only first 500 characters as a preview
        print(f"Best match is in document: {best_doc_name}")
        print(f"Content preview: {best_doc_text}...")

        # Add the question and result to the conversation history
        conversation_manager.add_to_history(query, best_doc_text)


if __name__ == "__main__":
    if 'ipykernel' in sys.modules or 'colab' in sys.modules:
        folder_path = "/content/data"
        main(folder_path)
    else:
        # Use typer for CLI usage in non-interactive environments
        typer.run(main)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

Processing PDFs...
Extracting text from CNT growth mechanism.pdf...
Extracting text from CNT growth functions.pdf...
Creating document embeddings...


  0%|          | 0/1 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 11.19 GiB. GPU 0 has a total capacity of 14.75 GiB of which 5.11 GiB is free. Process 10202 has 9.64 GiB memory in use. Of the allocated memory 9.16 GiB is allocated by PyTorch, and 365.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)